Let's demo how to calculate K corrections using Blanton for a single galaxy

In [1]:
import os
import argparse
import pandas as pd
from datasets import load_dataset
from tqdm import tqdm
import kcorrect.kcorrect
from astropy.io import fits

In [2]:
def load_astroclip_dataset(dataset_path, split):
    """Load the specified split of the AstroCLIP dataset."""
    print(f"Loading AstroCLIP dataset from: {dataset_path} (split: {split})")
    return load_dataset(dataset_path, split=split)
def load_photometry_data(file_path, key='df'):
    """Load photometry data from an HDF5 file."""
    print(f"Loading photometry data from: {file_path}")
    return pd.read_hdf(file_path, key=key)

In [3]:
astroclip_train = load_astroclip_dataset("/home/jrodriguez/deep-KCorrect/data/raw/AstroCLIP", split="train")
astroclip_test = load_astroclip_dataset("/home/jrodriguez/deep-KCorrect/data/raw/AstroCLIP", split="test")

Loading AstroCLIP dataset from: /home/jrodriguez/deep-KCorrect/data/raw/AstroCLIP (split: train)


Resolving data files:   0%|          | 0/139 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/36 [00:00<?, ?it/s]

Loading dataset shards:   0%|          | 0/92 [00:00<?, ?it/s]

Loading AstroCLIP dataset from: /home/jrodriguez/deep-KCorrect/data/raw/AstroCLIP (split: test)


Resolving data files:   0%|          | 0/139 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/36 [00:00<?, ?it/s]

Loading dataset shards:   0%|          | 0/24 [00:00<?, ?it/s]

In [4]:
photometry = load_photometry_data("/home/jrodriguez/deep-KCorrect/data/desi_edr_fluxes.h5")

Loading photometry data from: /home/jrodriguez/deep-KCorrect/data/desi_edr_fluxes.h5


In [5]:
def calculate_k_corrections_for_target(targetid, redshift, photometry, kc):
    """Calculate K-corrections and absolute magnitudes for a specific target."""
    results = {}
    filtered_df = photometry[photometry['targetid'] == targetid]
    if filtered_df.empty:
        raise ValueError(f"No data found for targetid: {targetid}")

    ivar = [filtered_df['flux_ivar_g'].item() * 10**18,
            filtered_df['flux_ivar_r'].item() * 10**18,
            filtered_df['flux_ivar_z'].item() * 10**18]
    dered_maggies = [filtered_df['dered_flux_g'].item() * 10**(-9),
                     filtered_df['dered_flux_r'].item() * 10**(-9),
                     filtered_df['dered_flux_z'].item() * 10**(-9)]

    coeffs = kc.fit_coeffs(redshift=redshift, maggies=dered_maggies, ivar=ivar)
    k_dered = kc.kcorrect(redshift=redshift, coeffs=coeffs, band_shift=0.1)
    
    results["K_G_G"] = k_dered[0]
    results["K_R_G"] = k_dered[1]
    results["K_Z_G"] = k_dered[2]
    results["K_R_R"] = k_dered[3]
    results["K_Z_R"] = k_dered[4]
    results["K_Z_Z"] = k_dered[5]

    absmag = kc.absmag(redshift=redshift, maggies=dered_maggies, ivar=ivar, coeffs=coeffs)
    results["absmag_g_g"] = absmag[0]
    results["absmag_g_r"] = absmag[1]
    results["absmag_g_z"] = absmag[2]
    results["absmag_r_r"] = absmag[3]
    results["absmag_z_r"] = absmag[4]
    results["absmag_z_z"] = absmag[5]
    results["z"] = redshift

    return results

In [6]:
# Initialize kcorrect
responses_in = ['bass_g', 'bass_r', 'mzls_z']
responses_out = ['sdss_g0', 'sdss_g0', 'sdss_g0', 'sdss_r0', 'sdss_r0', 'sdss_z0']
responses_map = ['bass_g', 'bass_r', 'mzls_z', 'bass_r', 'mzls_z', 'mzls_z']
kc = kcorrect.kcorrect.Kcorrect(responses=responses_in, responses_out=responses_out, responses_map=responses_map, abcorrect=False)

In [7]:
offending_target_id = 39633308409138239

In [23]:
calculate_k_corrections_for_target(39633422527760933,0.2222, photometry, kc)

{'K_G_G': 0.3159285804964115,
 'K_R_G': -0.9699822403249246,
 'K_Z_G': -1.9846507308255674,
 'K_R_R': -0.0006162519794707339,
 'K_Z_R': -1.0152847424801135,
 'K_Z_Z': -0.02442768609826772,
 'absmag_g_g': -19.419577,
 'absmag_g_r': -19.41958,
 'absmag_g_z': -19.419579,
 'absmag_r_r': -20.325846,
 'absmag_z_r': -20.325846,
 'absmag_z_z': -21.220984,
 'z': 0.2222}